# RNN 简介

# 时间序列模型概念简介
循环神经网络（RNN）是一种神经网络类型，其神经元的输出在下一个时间步会反馈作为输入，使网络具有处理序列数据的能力。它能处理变长序列，挖掘数据中的时序信息，不过存在长期依赖问题，即难以处理长序列中相距较远的信息关联。
RNN与普通神经网络的主要区别在于其具有记忆功能，神经元的输出能作为下一步输入，可处理序列数据，且输入和输出长度不固定；普通神经网络一般处理独立同分布的数据，层与层之间是简单的前馈连接关系，输入输出的长度通常是固定的。

RNN的应用场景广泛，在自然语言处理方面，可用于语言模型来预测下一个单词的概率，还能完成机器翻译、文本生成任务；在语音识别领域，能够处理语音这种时间序列信号，提高识别准确率；在时间序列预测中，像股票价格预测、天气预测等，RNN通过学习历史数据模式预测未来值；在视频分析中，它可以处理视频帧序列，进行动作识别等操作。





# RNN网络结构图
![图1](images/rnn.png)

RNN公式：
$$
[
\boldsymbol{h}_t = tanh(\boldsymbol{h}_{t-1} \boldsymbol{W}_h + \boldsymbol{x}_t \boldsymbol{W}_x + \boldsymbol{b})
]
$$

# 观察torch.nn.RNN的输入输出

In [79]:
import torch
import torch.nn as nn

In [114]:
# 单向、单层rnn
# 1个时间步
# batch_first=True表示输入数据的维度为[batch_size, seq_len, input_dim], input_dim在后文也称为input_size
single_rnn = nn.RNN(input_size=4, hidden_size=3, num_layers=1, batch_first=True) 
input = torch.randn(1, 1, 4) # 输入数据维度为[batch_size, time_steps_num, input_dim]
output, h_n = single_rnn(input) # output维度为[batch_size, time_steps_num, hidden_size=3]，h_n维度为[num_layers=1, batch_size, hidden_size=3]
print(input,output, output.shape, h_n, h_n.shape,  sep='\n')

tensor([[[ 0.2211,  0.0713, -0.7325,  0.2592]]])
tensor([[[-0.4568, -0.2468,  0.2100]]], grad_fn=<TransposeBackward1>)
torch.Size([1, 1, 3])
tensor([[[-0.4568, -0.2468,  0.2100]]], grad_fn=<StackBackward0>)
torch.Size([1, 1, 3])


In [113]:
# 单向、单层rnn
# 2个时间步
single_rnn = nn.RNN(input_size=4, hidden_size=3, num_layers=1, batch_first=True) # 输入数据的维度为[batch_size, time_steps_num, input_dim]
input = torch.randn(1, 2, 4) # 输入数据维度为[batch_size, time_steps_num, input_dim]
output, h_n = single_rnn(input) # output维度为[batch_size, time_steps_num, hidden_size=3]，h_n维度为[num_layers=1, batch_size, hidden_size=3]
print(input,output, output.shape, h_n, h_n.shape,  sep='\n')

tensor([[[ 1.3055,  0.0048, -0.0629,  0.4583],
         [ 0.8229,  1.5621,  0.1653,  0.5145]]])
tensor([[[-0.2296, -0.4769, -0.0592],
         [ 0.0134, -0.5053, -0.6914]]], grad_fn=<TransposeBackward1>)
torch.Size([1, 2, 3])
tensor([[[ 0.0134, -0.5053, -0.6914]]], grad_fn=<StackBackward0>)
torch.Size([1, 1, 3])


output输出为不同时间步的隐状态

In [82]:
# 双向、单层rnn
bi_rnn = nn.RNN(input_size=4, hidden_size=3, num_layers=1, batch_first=True, bidirectional=True)
bi_output, bi_h_n = bi_rnn(input)
print(bi_output, bi_output.shape, bi_h_n, bi_h_n.shape, sep='\n')

tensor([[[ 0.0293, -0.2261,  0.3965, -0.8093,  0.5495, -0.2421],
         [ 0.3172, -0.4707, -0.0548, -0.9589,  0.5515,  0.0759],
         [-0.3819, -0.9026,  0.2700, -0.6062,  0.9286, -0.6791],
         [-0.9650,  0.2898,  0.9175, -0.9964,  0.3749, -0.4732],
         [ 0.4947, -0.6497,  0.0801, -0.3799,  0.8914, -0.4917]],

        [[ 0.1236,  0.6172,  0.5129, -0.9334, -0.7831,  0.1077],
         [ 0.7416,  0.5501,  0.4543, -0.8432, -0.2094, -0.3928],
         [ 0.9069, -0.6283, -0.4312, -0.5202,  0.6983, -0.2993],
         [ 0.2843, -0.9798, -0.5583, -0.0776,  0.9733,  0.1556],
         [-0.9714, -0.1158,  0.7961, -0.9926,  0.1743,  0.1932]],

        [[ 0.8565, -0.8896, -0.7905, -0.4024,  0.6848,  0.4695],
         [-0.2559,  0.0835,  0.7091, -0.7468, -0.3244, -0.6832],
         [-0.3923, -0.4974,  0.4001, -0.9646,  0.8942,  0.0540],
         [ 0.2724, -0.8785, -0.4926, -0.8918,  0.8703,  0.0652],
         [ 0.4889, -0.8752, -0.3374,  0.1035,  0.6077, -0.4534]]],
       grad_fn=<Tra

# 从零手搓 RNN 

### 自定义单向单层RNN Layer

In [83]:
import torch
import torch.nn as nn

对照RNN公式实现RNN Layer
$$
[
\boldsymbol{h}_t = tanh(\boldsymbol{h}_{t-1} \boldsymbol{W}_h + \boldsymbol{x}_t \boldsymbol{W}_x + \boldsymbol{b})
]
$$

In [84]:
class RNNLayer(nn.Module):
    def __init__(self,input_size, hidden_size, num_layers=1, batch_first=True):
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.input_size = input_size
        self.bidirectional = False
        super().__init__()
        self.W_ih = nn.Parameter(torch.rand(self.input_size, self.hidden_size))
        self.W_hh = nn.Parameter(torch.rand(self.hidden_size, self.hidden_size))
        self.b_ih = nn.Parameter(torch.zeros(self.hidden_size))
        self.b_hh = nn.Parameter(torch.zeros(self.hidden_size))
        
    def forward(self,x_t,h_prev=None):
        # part 1: torch.matmul(x_t, self.W_ih)
        # x_t包含多个时间步，形状为[batch_size, time_steps_num, input_dim]
        # W_ih形状为[input_dim, hidden_size]
        # torch.matmul(x_t, self.W_ih) 输出矩阵形状为[batch_size, time_steps_num, hidden_size]
        # part 2: torch.matmul(h_prev, self.W_hh)
        # h_prev 形状为[batch_size, time_steps_num, hidden_size]
        # W_hh形状为[hidden_size, hidden_size]
        # torch.matmul(h_prev, self.W_hh) 输出矩阵形状为[batch_size, time_steps_num, hidden_size]
        if h_prev == None:
             h_prev = torch.zeros( x_t.size(0), self.hidden_size)
        output = torch.tanh(torch.matmul(x_t, self.W_ih) + self.b_ih + torch.matmul(h_prev, self.W_hh) + self.b_hh)
        return output,output[:,-1,:].unsqueeze(0)
        

### 测试输出

In [193]:
# 单向、单层rnn
single_rnn = RNNLayer(input_size=4, hidden_size=3, num_layers=1, batch_first=True) # batch_first=True表示输入数据的维度为[batch_size, time_steps_num, input_dim]
input = torch.randn(1, 5, 4) # 输入数据维度为[batch_size, time_steps_num, input_size]
output,h_n = single_rnn(input) # output维度为[batch_size, time_steps_num, hidden_size=3]，h_n维度为[num_layers=1, batch_size, hidden_size=3]
print(output, output.shape, h_n, h_n.shape,  sep='\n')

tensor([[[ 0.9629,  0.9930,  0.7752],
         [-0.2752, -0.1178, -0.2255],
         [-0.4656, -0.5441, -0.2772],
         [-0.7855, -0.8322,  0.0315],
         [ 0.7842,  0.9167,  0.8217]]], grad_fn=<TanhBackward0>)
torch.Size([1, 5, 3])
tensor([[[0.7842, 0.9167, 0.8217]]], grad_fn=<UnsqueezeBackward0>)
torch.Size([1, 1, 3])


输出结果形状与nn.RNN一致

### 用nn.RNN建立模型

In [208]:
import torch.nn.functional as F

In [201]:
class CustomRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.vocab_size = output_size# 输入是One hot, output_size和vocab_size 都是词表大小
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, X):
        # 这里的X输入为word index
        X = F.one_hot(torch.tensor(torch.tensor(X)),self.vocab_size)
        X = X.to(torch.float32)
        print(X.size())
        state_0 = torch.zeros(self.num_layers, X.size(0), self.hidden_size).to(X.device) # 隐状态的形状为[层数，batch_size,hidden_size]
        out, state = self.rnn(X, state_0) 
        out = self.fc(out[:, -1, :])  # 取最后一个时间步的输出
        return out

### 测试模型输出

首先导入数据集

In [154]:
def load_data_lyrics():
    #with zipfile.ZipFile('./test.txt') as zin:
    with open('test.txt') as f:
            corpus_chars = f.read()#.decode('utf-8')
    # corpus_chars[:40]  # '想要有直升机\n想要和你飞到宇宙去\n想要和你融化在一起\n融化在宇宙里\n我每天每天每'

    # 将换行符替换成空格；仅使用前1万个字符来训练模型
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]

    # 将每个字符映射成索引
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)  # 1027
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    sample = corpus_indices[:20]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size


In [155]:
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_lyrics()

测试模型输出

In [197]:
model = CustomRNN(256, 10,1,256)
Y = model([[13,12,14]])

torch.Size([1, 3, 256])


/tmp/ipykernel_46949/1910301334.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = F.one_hot(torch.tensor(torch.tensor(X)),self.vocab_size)


In [198]:
Y.size()

torch.Size([1, 256])

In [164]:
index = Y.argmax(dim=1)

In [165]:
idx_to_char[index]

'动'

把推理部分打包成函数

In [199]:
def predict(init_chars,model,time_steps_num,idx_to_char,char_to_idx):
    X = []
    for c in init_chars:
        X.append(char_to_idx[c])
    output = init_chars
    print(X)
    for i in range(time_steps_num):
        Y= model([X])
        idx = Y.argmax(dim=1)
        X.append(idx)
        output+=idx_to_char[idx]
    return output

In [202]:
predict('构', model,5,idx_to_char,char_to_idx)

[12]
torch.Size([1, 1, 256])
torch.Size([1, 2, 256])
torch.Size([1, 3, 256])
torch.Size([1, 4, 256])
torch.Size([1, 5, 256])


/tmp/ipykernel_46949/1910301334.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = F.one_hot(torch.tensor(torch.tensor(X)),self.vocab_size)


'构0G0G0'

模型尚未训练，输出随机结果

### 用自定义的RNN Layer预测

In [209]:
class CustomRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.vocab_size = output_size# 输入是One hot, output_size和vocab_size 都是词表大小
        self.num_layers = num_layers
        self.rnn = RNNLayer(input_size, hidden_size, num_layers, batch_first=True)  # 讲nn.RNN替换为自定义的RNNLayer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, X):
        # 这里的X输入为word index
        X = F.one_hot(torch.tensor(torch.tensor(X)),self.vocab_size)
        X = X.to(torch.float32)
        print(X.size())
        state_0 = torch.zeros(self.num_layers, X.size(0), self.hidden_size).to(X.device) # 隐状态的形状为[层数，batch_size,hidden_size]
        out, state = self.rnn(X, state_0) 
        out = self.fc(out[:, -1, :])  # 取最后一个时间步的输出
        return out

In [204]:
model = CustomRNN(256, 10,1,256)
predict('构', model,5,idx_to_char,char_to_idx)

[12]
torch.Size([1, 1, 256])
torch.Size([1, 2, 256])
torch.Size([1, 3, 256])
torch.Size([1, 4, 256])
torch.Size([1, 5, 256])


/tmp/ipykernel_46949/1910301334.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = F.one_hot(torch.tensor(torch.tensor(X)),self.vocab_size)


'构形h端回h'

同样也是随机结果

### 思考：
- 如何构建多层、双向的RNN？
- 如何使用自定义的RNN Layer及模型训练一个歌词或者诗词生成器？